<a href="https://www.kaggle.com/code/w1ndness/data-analysis-08?scriptVersionId=110454597" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tourism-data/test/test.xlsx
/kaggle/input/tourism-data/test/result1.csv
/kaggle/input/tourism-data/train/2018-2019.xlsx
/kaggle/input/tourism-data/train/2020-2021.xlsx


In [2]:
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import jieba
import re
%matplotlib inline
warnings.filterwarnings('ignore')

# Task 1

In [3]:
data1 = pd.read_excel('/kaggle/input/tourism-data/train/2018-2019.xlsx', sheet_name=4)
data2 = pd.read_excel('/kaggle/input/tourism-data/train/2020-2021.xlsx', sheet_name=4)
test_data = pd.read_excel('../input/tourism-data/test/test.xlsx')

In [4]:
data1.head()

,文章ID,公众号标题,发布时间,正文
0,1001,2018，对自己好一点,2018-01-02 17:28,2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，请对自己好一点\n一辈子很...
1,1002,春节机票预订有窍门,2018-01-02 17:28,距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚至一票难求。在这里分享...
2,1003,冬日旅游知多D,2018-01-03 17:32,960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\n但是冬季的严寒气...
3,1004,2018冬季暖心之旅,2018-01-03 17:32,长按二维码，关注我们\n中心联系人：林小姐13709649096\n刘小姐135000781...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明,2018-01-05 16:57,


In [5]:
data2.head()

,文章ID,公众号标题,发布时间,正文
0,1622,元旦晚8点，茂名海鲜亮相央视！一定要看！,2020-01-01 19:22,新年伊始，你的打开方式是怎样的呢？是准备去享受一顿丰盛的大餐？还是将“世界那么大，我想去看看...
1,1623,建好省级全域旅游示范区，打造沿海经济带新增长极 茂名市电白区旅游协会举行第二届第二次全体会议,2020-01-05 02:10,会员报到\n\n会场\n\n 为了更好地贯彻落实区委区政府和区文广旅体局建设全...
2,1624,【茂名交投旅游】@茂名人，信宜旅游景点上新啦~,2020-01-06 17:27,好消息！好消息！\n\n\n信宜莲花湖庄园\n\n开业啦~~~\n庄园简介\n\n\n\n\...
3,1625,【茂名交投旅游】2020，打卡鼎龙湾新玩法！,2020-01-06 17:27,2020.首波惊喜狂潮\n鼎龙湾·德萨斯牛仔小镇全面升级\n1月11日\n牛仔主题嘉年华街区...
4,1626,茂名 | 串珠成链，建滨海旅游目的地,2020-01-08 21:35,地处粤港澳大湾区、北部湾城市群、海南自贸区三大国家战略交汇处的茂名，紧紧抓住沿海经济带与大湾...


In [6]:
columns = ['ID', 'Title', 'Date', 'Content']
data1.columns = columns
data2.columns = columns

In [7]:
data = pd.concat([data1, data2], ignore_index=True)
data['Content'] = data['Title'] + '\n' + data['Content']
data.drop(columns=['Title', 'Date'], inplace=True)
data.head()

,ID,Content
0,1001,2018，对自己好一点\n2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，...
1,1002,春节机票预订有窍门\n距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚...
2,1003,冬日旅游知多D\n960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\...
3,1004,2018冬季暖心之旅\n长按二维码，关注我们\n中心联系人：林小姐13709649096\n...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明\n ...


In [8]:
data.dropna(inplace=True)

In [9]:
test_data.head()

,文章ID,公众号标题,正文
0,8396,@茂名家长，广东全面启动3-11岁儿童新冠疫苗接种！是否安全？最全问答来了→,按照国家整体部署\n广东省全面启动\n3-11岁人群新冠病毒疫苗接种\n3-11岁人群接种哪...
1,8397,庄悦群：把安全生产摆在首要位置抓紧抓实，加快推进绿色矿山智慧矿山建设,10月31日上午，市委副书记、市长庄悦群带队到高州市，就深入贯彻落实习近平总书记关于安全生产...
2,8398,群众办事百项堵点疏解行动,为破解群众办事堵点难点，发展改革委会同新华社、中国政府网于4月19日启动了“群众办事百项...
3,8399,茂名启动台风Ⅲ级应急响应！市旅游局发布防御台风安全提示,点击上方蓝字，关注我们\n\n\n台风最新消息\n \n目前，台风正朝着广东袭来\n↓↓↓\...
4,8400,灾情就是命令,受南海热带低压影响，我市近日受强降雨影响，钱排、合水、新宝、平塘、思贺等镇遭受特大暴雨袭击，...


In [10]:
test_data.columns = ['ID', 'Title', 'Content']
test_data['Content'] = test_data['Title'] + '\n' + test_data['Content']
test_data.drop(columns=['Title'], inplace=True)
test_data.head()

,ID,Content
0,8396,@茂名家长，广东全面启动3-11岁儿童新冠疫苗接种！是否安全？最全问答来了→\n按照国家整体...
1,8397,庄悦群：把安全生产摆在首要位置抓紧抓实，加快推进绿色矿山智慧矿山建设\n10月31日上午，市...
2,8398,群众办事百项堵点疏解行动\n 为破解群众办事堵点难点，发展改革委会同新华社、中国政府网于4...
3,8399,茂名启动台风Ⅲ级应急响应！市旅游局发布防御台风安全提示\n点击上方蓝字，关注我们\n\n\n...
4,8400,灾情就是命令\n受南海热带低压影响，我市近日受强降雨影响，钱排、合水、新宝、平塘、思贺等镇遭...


In [11]:
print('Train data shape:', data.shape)
print('Test data shape:', test_data.shape)
train_size = data.shape[0]

Train data shape: (6285, 2)
Test data shape: (930, 2)


In [12]:
data = pd.concat([data, test_data], ignore_index=True)
data.shape

(7215, 2)

In [13]:
def preprocessing(data):
    try:
        lines = data.split()
    except Exception as e:
        print(data)
        return ''
    lines = list(filter(lambda x: x is not '', lines))
    unuse_lis = []
    rule_1 = r'\W'
    compiled_rule_1 = re.compile(rule_1)
    for line in lines:
        no_en_and_da = compiled_rule_1.findall(line)
        no_en_and_da_str = ''.join(no_en_and_da)
        reslis = re.findall(r'^\S', ''.join(re.findall(r'[^\，]', ''.join(re.findall(r'[^\。]', no_en_and_da_str)))))
        unuse_lis.append(reslis)
    syms = []
    for i in unuse_lis:
        for j in i:
            syms.append(j)
    syms = list(set(syms))
    
    def replace_syms(line):
        for sym in syms:
            line = line.replace(sym, '')
        return line
    
    def replace_lem(line):
        a = re.sub(r'\s', '', line)
        b = re.sub(r'\W{2,}', '', a)
        c = re.sub(r'\d', '', b)
        d = re.sub(r' ', '', c)
        d = d.replace('_', '')
        return d
    
    lines = list(map(replace_syms, lines))
    lines = list(map(replace_lem, lines))
    lines = list(filter(lambda x: x not in '1234567890', lines))
    
    res = []
    for line in lines:
        for word in jieba.cut(line):
            if word == '，':
                continue
            res.append(word)
    return ' '.join(res)

In [14]:
preprocessing(data.loc[0, 'Content'])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.837 seconds.
Prefix dict has been built successfully.


'对 自己 好 一点 的 旅程 已经 结束 的 未来 拉开 了 帷幕 新 的 一年 里 请 对 自己 好 一点 一辈子 很长 漫漫 人生 路上 会 遇到 那么 多 的 人和事 一辈子 很 短 一年 一年 时光 悄悄 从 指缝间 溜走 在 有限 的 生命 里 从 现在 开始 享受 自己 的 人生 不 强求 缘分 该 遇见 的 人 总会 遇见 留不住 的 过客 终将 远走 有缘 相守 心怀 感恩 无缘 结交 坦然 放手 不 强求 你 我 都 是 自由 的 随缘 就 好 陪伴 家人 站 在 我们 身后 的 家人 是 前进 时 的 坚强 后盾 给予 我们 无条件 的 支持 是 孤独 无助 时 的 避风港 默默 付出 而 不求 回报 把 时间 留 一点 给 家人 他们 所 需 的 不过 是 陪伴 珍惜 朋友 时光流逝 仿佛 大浪淘沙 留下来 的 真心 朋友 不多不少 刚刚 好 三五 知己 小酌 几杯 君子之交 虽淡 但 贵在 真心 足矣 认真 工作 新 的 一年 里 全身心 投入 工作 付出 会 有 回报 工作 会为 你 搭 起 梦想 与 人生价值 之间 的 桥梁 认真 工作 别 辜负 自己 爱护 身体 健康 的 体魄 是 自己 最大 的 本钱 闲暇 时 散散步 锻炼 一下 身体 用 自律 而 健康 的 作息 对 自己 的 身体 负责 出去 走走 心灵 也 需要 放松 与 远行 城市 里 的 拥挤 高楼 令人 迷茫 不妨 出去 走走 行万里路 感受 万里 风光 体验 形色 人生 过去 已经 过去 未来 即将 到来 新 的 一年 愿 你 想要 的 明天 都 终将 到来 更 多 精彩 线路 欢迎 关注 茂名 交投 旅游 集散中心 站 前 四路 交委 车站 一楼 中心 联系人 林小姐 刘小姐 罗 小姐 陈小姐 市区 联系人 柯 小姐 柯 先生 吴小姐 梁小姐 邓 先生 电白 分部 蔡 小姐 高州 分部 唐小姐 化州 分部 马先生 李小姐 信宜 分部 陈小姐 谢小姐'

In [15]:
data['Cut'] = data['Content'].apply(preprocessing)
data.head()

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


,ID,Content,Cut
0,1001,2018，对自己好一点\n2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，...,对 自己 好 一点 的 旅程 已经 结束 的 未来 拉开 了 帷幕 新 的 一年 里 请 对...
1,1002,春节机票预订有窍门\n距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚...,春节 机票 预订 有 窍门 距离 春节 还有 一个多月 的 时间 在线 旅游 网站 的 春节...
2,1003,冬日旅游知多D\n960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\...,冬日 旅游 知多 D 万平方公里 的 祖国 大地 四季 都 有 独特 美景 冬天 的 旅行 ...
3,1004,2018冬季暖心之旅\n长按二维码，关注我们\n中心联系人：林小姐13709649096\n...,冬季 暖心 之 旅 长 按 二维码 关注 我们 中心 联系人 林小姐 刘小姐 罗 小姐 陈小...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明\n ...,关于 粤 K 号 大客车 排气管 喷火 ” 事件 的 情况 说明


In [16]:
data.shape

(7215, 3)

## 1.1 By LDA

In [17]:
# data['AsList'] = data['Cut'].apply(lambda x: list(filter(lambda x: ' ' not in x, jieba.lcut(x))))
# data['AsList'].head()

In [18]:
# text_data = data['AsList'].tolist()

In [19]:
# from gensim import corpora
# from gensim.models import TfidfModel

# dictionary = corpora.Dictionary(text_data)
# dictionary.filter_n_most_frequent(200)
# corpus = [dictionary.doc2bow(text) for text in text_data]

# # tfidf = TfidfModel(corpus)
# # tfidf.save('task_1_tfidf.model')
# # # corpus = tfidf[corpus]

In [20]:
# dictionary.save('task_1.dict')

In [21]:
# from gensim.models import LdaModel

# # Set training parameters.
# num_topics = 10
# chunksize = 2000
# passes = 20
# iterations = 200
# eval_every = None  # Don't evaluate model perplexity, takes too much time.

# # Make a index to word dictionary.
# temp = dictionary[0]  # This is only to "load" the dictionary.
# id2word = dictionary.id2token

# model = LdaModel(
#     corpus=corpus,
#     id2word=id2word,
#     chunksize=chunksize,
#     alpha='auto',
#     eta='auto',
#     iterations=iterations,
#     num_topics=num_topics,
#     passes=passes,
#     eval_every=eval_every
# )

# model.save('task_1.model')  # 将模型保存到硬盘

In [22]:
# topic_list = model.print_topics()
# topic_list = sorted(topic_list, key=lambda x: x[0])
# print(topic_list)

In [23]:
# top_topics = model.top_topics(corpus)

# # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
# avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
# print('Average topic coherence: %.4f.' % avg_topic_coherence)

# # from pprint import pprint
# # pprint(top_topics)

In [24]:
# lda_topics = []
# for topic in top_topics:
#     topic_list, _ = topic
#     lda_topics.append([x[1] for x in topic_list])
# lda_topics

In [25]:
# test_doc = text_data[23]
# print(test_doc)
# doc_bow = dictionary.doc2bow(test_doc)
# # doc_tfidf = tfidf[doc_bow]
# doc_lda = model[doc_bow]
# doc_lda

In [26]:
# def judge_coherence(doc):
#     coh_dict = ['旅游', '活动', '节庆', '特产', '交通', '酒店', '景区', '景点',
#                 '文创', '文化', '乡村旅游', '民宿', '假日', '假期', '游客', '采摘',
#                 '赏花', '春游', '踏青', '康养', '公园', '滨海游', '度假', '农家乐',
#                 '剧本杀', '旅行', '徒步', '工业旅游', '线路', '自驾游', '团队游',
#                 '攻略', '游记', '包车', '玻璃栈道', '游艇', '高尔夫', '温泉']
#     doc_bow = dictionary.doc2bow(test_doc)
# #     doc_tfidf = tfidf[doc_bow]
#     doc_lda = model[doc_bow]
#     topic_idx = [each[0] for each in doc_lda]
#     topics = []
#     for idx in topic_idx:
#         topics.extend(lda_topics[idx - 1])
    
#     def judge(topics, coh_dict):
#         for x in topics:
#             for y in coh_dict:
#                 if y in x:
#                     return True
#         return False
    
#     return judge(topics, coh_dict)

In [27]:
# test_text = text_data[train_size:]
# test_res = [judge_coherence(text) for text in test_text]

## 1.2 By TF-IDF and TextRank

In [28]:
import jieba.analyse

def judge_by_tfidf(text):
    coh_dict = ['旅游', '活动', '节庆', '特产', '交通', '酒店', '景区', '景点',
                '文创', '文化', '乡村旅游', '民宿', '假日', '假期', '游客', '采摘',
                '赏花', '春游', '踏青', '康养', '公园', '滨海游', '度假', '农家乐',
                '剧本杀', '旅行', '徒步', '工业旅游', '线路', '自驾游', '团队游',
                '攻略', '游记', '包车', '玻璃栈道', '游艇', '高尔夫', '温泉']
    keywords = jieba.analyse.extract_tags(text, topK=50, withWeight=False)
    for x in keywords:
        for y in coh_dict:
            if y in x:
                return True
    return False

def judge_by_textrank(text):
    coh_dict = ['旅游', '活动', '节庆', '特产', '交通', '酒店', '景区', '景点',
                '文创', '文化', '乡村旅游', '民宿', '假日', '假期', '游客', '采摘',
                '赏花', '春游', '踏青', '康养', '公园', '滨海游', '度假', '农家乐',
                '剧本杀', '旅行', '徒步', '工业旅游', '线路', '自驾游', '团队游',
                '攻略', '游记', '包车', '玻璃栈道', '游艇', '高尔夫', '温泉']
    keywords = jieba.analyse.textrank(text, topK=50, withWeight=False)
    for x in keywords:
        for y in coh_dict:
            if y in x:
                return True
    return False

def judge(text):
    return '相关' if judge_by_tfidf(text) and judge_by_textrank(text) else '不相关'

test_res = [judge(text) for text in data[train_size:]['Cut'].tolist()]

In [29]:
task1_res = pd.DataFrame({'文章ID': test_data['ID'].tolist(), 
                          '分类标签': test_res})
task1_res.to_csv('result1.csv', index=False)

# Task 2

## Task 2.1

In [30]:
def read_in(path):
    sheet0 = pd.read_excel(path, sheet_name=0)
    sheet1 = pd.read_excel(path, sheet_name=1)
    sheet2 = pd.read_excel(path, sheet_name=2)
    sheet3 = pd.read_excel(path, sheet_name=3)
    sheet0.columns = ['ID', 'City', 'Name', 'RevDate', 'Content', 'CheckDate', 'HouseType']
    sheet0['ID'] = sheet0['ID'].apply(lambda x: '酒店评论-' + str(x))
    sheet0 = sheet0[['ID', 'Content']]
    sheet1.columns = ['ID', 'City', 'Name', 'Date', 'Content']
    sheet1['ID'] = sheet1['ID'].apply(lambda x: '景区评论-' + str(x))
    indices = sheet1[sheet1['Name'].str.contains('湛江|广东海洋大学|南极长城站')].index
    sheet1.drop(index=indices, inplace=True)
    sheet1 = sheet1[['ID', 'Content']]
    sheet2 = sheet2[['游记ID', '正文']]
    sheet2.columns = ['ID', 'Content']
    sheet2['ID'] = sheet2['ID'].apply(lambda x: '游记-' + str(x))
    sheet3 = sheet3[['餐饮评论ID', '评论内容', '标题']]
    sheet3.columns = ['ID', 'Content', 'Title']
    sheet3['Content'] = sheet3['Title'] + '\n' + sheet3['Content']
    sheet3.drop(columns='Title', inplace=True)
    sheet3['ID'] = sheet3['ID'].apply(lambda x: '餐饮评论-' + str(x))
    ret = pd.concat([sheet0, sheet1, sheet2, sheet3], ignore_index=True)
    return ret

In [31]:
data1 = read_in('../input/tourism-data/train/2018-2019.xlsx')
data2 = read_in('../input/tourism-data/train/2020-2021.xlsx')
rev_data = pd.concat([data1, data2], ignore_index=True)
rev_data.head(), rev_data.shape

(          ID                                            Content
 0  酒店评论-1001                                            干净卫生服务好
 1  酒店评论-1002                                           环境可以，干净！
 2  酒店评论-1003  环境不错，房间卫生都很好，生活也很方便，就是隔音效果不理想，有时太吵。我定的优惠价，性价比很...
 3  酒店评论-1004                                    很好.......舒服态度不错
 4  酒店评论-1005                                 #卫生# #设计风格# #酒店餐饮#,
 (9496, 2))

In [32]:
rev_data['Content'] = rev_data['Content'].apply(lambda x: re.sub(r'^.*?\n\d+\-\d+\-\d+.*?\nhttp.*?\n|\n.*?\d+\-\d+\-\d+.*?\nhttp.*?\n|\d+\-\d+\-\d+.*?\nhttp.*?\n| ', '', x))
rev_data['Content'].dropna()
rev_data.head(), rev_data.shape

(          ID                                            Content
 0  酒店评论-1001                                            干净卫生服务好
 1  酒店评论-1002                                           环境可以，干净！
 2  酒店评论-1003  环境不错，房间卫生都很好，生活也很方便，就是隔音效果不理想，有时太吵。我定的优惠价，性价比很...
 3  酒店评论-1004                                    很好.......舒服态度不错
 4  酒店评论-1005                                   #卫生##设计风格##酒店餐饮#,
 (9496, 2))

In [33]:
!pip install pyhanlp
!pip install foolnltk
!pip install tensorflow==1.14

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 896.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyhanlp: filename=pyhanlp-0.1.84-py3-none-any.whl size=29818 sha256=5fef7ef20d83ad73ea83df65695a76d73d7e17cc0d476355dd1aff2a435b6b8d
  Stored in directory: /root/.cache/pip/wheels/8e/67/74/f1dfe4d6ea994941a47aee14de63ca71aba2ab96e2e217c072
  Created wheel for hanlp-downloader: filename=hanlp_downloader-0.0.25-py3-none-any.whl size=13776 sha256=565ca204e4a90dc143d2fe2837acc798260aecc82fefa1145f35079b8f194df6
  Stored in directory: /root/.cache/pip/wheels/cb/a2/c3/61f7b80f5d56e5b19dd9cb50303f8f3b621662f035a281d197
Successfully built pyhanlp hanlp-downloader
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [34]:
from pyhanlp import *
import fool

rev_data['Content'] = rev_data['Content'].apply(HanLP.convertToSimplifiedChinese)

下载 https://file.hankcs.com/hanlp/hanlp-1.8.3-release.zip 到 /opt/conda/lib/python3.7/site-packages/pyhanlp/static/hanlp-1.8.3-release.zip
100%   1.8 MiB   1.4 MiB/s ETA:  0 s [=========================================]
下载 https://file.hankcs.com/hanlp/data-for-1.7.5.zip 到 /opt/conda/lib/python3.7/site-packages/pyhanlp/static/data-for-1.8.3.zip
100% 637.7 MiB  22.4 MiB/s ETA:  0 s [=========================================]


解压 data.zip...


In [35]:
task2_1 = pd.DataFrame(columns=['ID', 'pID', 'Name'])
entity_data = pd.DataFrame(columns=['ID', 'Entities'])
entity_dict = {}
r = 1
cnt = 1

def judge(entity: str):
    if str.__contains__(entity, '国') or str.__contains__(entity, '市') or str.__contains__(entity, '区') or str.__contains__(entity, '州'):
        return False
    if entity in ['茂名', '湛江', '河东', '粤西', '水东']:
        return False
    if entity in ['河北', '山西', '辽宁', '吉林', '黑龙江', 
                  '江苏', '浙江', '安徽', '福建', '江西', 
                  '山东', '河南', '湖北', '湖南', '广东', 
                  '海南', '四川', '贵州', '云南', '陕西', 
                  '甘肃', '青海', '台湾', '内蒙古', '广西', 
                  '西藏', '宁夏', '新疆', '北京', '天津', 
                  '上海', '重庆', '香港', '澳门']:
        return False
    return True

for index, row in rev_data.iterrows():
    ID, text = row['ID'], row['Content']
    res = fool.analysis(text)[1][0]
    if res != []:
        entities = []
        for each in res:
            _, _, type_, entity = each
            if type_ == 'location' and judge(entity):
                entity = entity.strip()
                entity = entity.replace(' ', '')
                entity = entity.replace('\n', '')
                entities.append(entity)
                if entity not in entity_dict:
                    task2_1.loc[cnt - 1, 'ID'] = ID
                    task2_1.loc[cnt - 1, 'pID'] = 'ID' + str(cnt)
                    task2_1.loc[cnt - 1, 'Name'] = entity
                    entity_dict[entity] = cnt
                    cnt += 1
                else:
                    pID = entity_dict[entity]
                    task2_1.loc[cnt - 1, 'ID'] = ID
                    task2_1.loc[cnt - 1, 'pID'] = 'ID' + str(pID)
                    task2_1.loc[cnt - 1, 'Name'] = entity
        entity_data.loc[r, 'ID'] = ID
        entity_data.loc[r, 'Entities'] = entities
        r += 1
task2_1.columns = ['语料ID', '产品ID', '产品名称']
task2_1.to_csv('result2-1.csv', index=False)

2022-11-09 02:08:21.133085: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-11-09 02:08:21.142251: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000179999 Hz
2022-11-09 02:08:21.142407: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55bfcfc5d900 executing computations on platform Host. Devices:
2022-11-09 02:08:21.142421: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-11-09 02:08:21.331493: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA

## Task 2.2

In [36]:
def read_in(path):
    sheet1 = pd.read_excel(path, sheet_name=0)
    sheet2 = pd.read_excel(path, sheet_name=1)
    sheet3 = pd.read_excel(path, sheet_name=3)
    sheet1['Label'] = '酒店'
    hotels = sheet1[['酒店名称', 'Label']]
    hotels.columns = ['Name', 'Label']
    sheet2['Label'] = '景区'
    views = sheet2[['景区名称', 'Label']]
    views.columns = ['Name', 'Label']
    sheet3['Label'] = '餐饮'
    meals = sheet3[['餐饮名称', 'Label']]
    meals.columns = ['Name', 'Label']
    ret = pd.concat([hotels, views, meals], ignore_index=False)
    return ret

In [37]:
data1 = read_in('../input/tourism-data/train/2018-2019.xlsx')
data2 = read_in('../input/tourism-data/train/2020-2021.xlsx')
train_data = pd.concat([data1, data2], ignore_index=True)
train_data.head(), train_data.shape

(             Name Label
 0        茂名君悦商务酒店    酒店
 1  维也纳国际酒店(茂名电白店)    酒店
 2          茂名永利之家    酒店
 3          茂名诚荟酒店    酒店
 4        茂名华景商务酒店    酒店,
 (9280, 2))

In [38]:
train_data['Name'] = train_data['Name'].astype('str')
train_data['Label'] = train_data['Label'].astype('str')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9280 entries, 0 to 9279
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    9280 non-null   object
 1   Label   9280 non-null   object
dtypes: object(2)
memory usage: 145.1+ KB


In [39]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train_data['Label'] = encoder.fit_transform(train_data['Label'])

In [40]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
import random
import re

# 划分为训练集和验证集
# stratify 按照标签进行采样，训练集和验证部分同分布
x_train, x_test, train_label, test_label =  train_test_split(train_data['Name'].tolist(), 
                      train_data['Label'].tolist(), test_size=0.3, stratify=train_data['Label'].tolist())

In [41]:
!pip install transformers==4.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires h5py>=3.6.0, but you have h5py 3.1.0 which is incompatible.
allennlp 2.10.1 requires numpy>=1.21.4, but you have numpy 1.19.5 which is incompatible.


In [42]:
os.environ["USE_TF"] = 'None'
# transformers bert相关的模型使用和加载
from transformers import BertTokenizer
# 分词器，词典

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_encoding = tokenizer(x_train, truncation=True, padding=True, max_length=64)
test_encoding = tokenizer(x_test, truncation=True, padding=True, max_length=64)

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [43]:
class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    # 读取单个样本
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encoding, train_label)
test_dataset = MyDataset(test_encoding, test_label)

In [44]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# 优化方法
optim = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 1
scheduler = get_linear_schedule_with_warmup(optim, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [45]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [46]:
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    for batch in train_loader:
        # 正向传播
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        total_train_loss += loss.item()
        
        # 反向梯度信息
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 参数更新
        optim.step()
        scheduler.step()

        iter_num += 1
        if(iter_num % 100==0):
            print("epoth: %d, iter_num: %d, loss: %.4f, %.2f%%" % (epoch, iter_num, loss.item(), iter_num / total_iter * 100))
        
    print("Epoch: %d, Average training loss: %.4f" % (epoch, total_train_loss / len(train_loader)))
    
def validation():
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    for batch in test_dataloader:
        with torch.no_grad():
            # 正常传播
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs[0]
        logits = outputs[1]

        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
    print("Accuracy: %.4f" % (avg_val_accuracy))
    print("Average testing loss: %.4f" % (total_eval_loss / len(test_dataloader)))
    print("-------------------------------")

In [47]:
num_epochs = 2

for epoch in range(num_epochs):
    print("------------Epoch: %d ----------------" % epoch)
    train()
    validation()

------------Epoch: 0 ----------------
epoth: 0, iter_num: 100, loss: 0.0024, 24.63%
epoth: 0, iter_num: 200, loss: 0.0011, 49.26%
epoth: 0, iter_num: 300, loss: 0.0009, 73.89%
epoth: 0, iter_num: 400, loss: 0.0006, 98.52%
Epoch: 0, Average training loss: 0.0425
Accuracy: 1.0000
Average testing loss: 0.0003
-------------------------------
------------Epoch: 1 ----------------
epoth: 1, iter_num: 100, loss: 0.0009, 24.63%
epoth: 1, iter_num: 200, loss: 0.0006, 49.26%
epoth: 1, iter_num: 300, loss: 0.0008, 73.89%
epoth: 1, iter_num: 400, loss: 0.0005, 98.52%
Epoch: 1, Average training loss: 0.0006
Accuracy: 1.0000
Average testing loss: 0.0003
-------------------------------


In [48]:
entities = list(entity_dict.keys())

entity_encoding = tokenizer(entities, truncation=True, padding=True, max_length=64)
entity_dataset = MyDataset(entity_encoding, np.zeros(len(entities)))
entity_types = []

entity_loader = DataLoader(entity_dataset, batch_size=1, shuffle=False)

for idx, batch in enumerate(entity_loader):
    with torch.no_grad():
        # 正常传播
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs[0]
        logits = outputs[1]
        logits = logits.detach().cpu().numpy()
        type_ = np.argmax(logits)
        entity_types.append(type_)
entity_types = encoder.inverse_transform(entity_types)

In [49]:
def evaluate_counter(entity, data):
    return len(data[data['Content'].str.contains(entity)].index)

entity_evaluate = []
for entity in entities:
    entity_evaluate.append(evaluate_counter(entity, rev_data))

In [50]:
def essay_counter(entity, data):
    return len(data[data['Cut'].str.contains(entity)].index)

entity_essay = []
for entity in entities:
    entity_essay.append(essay_counter(entity, data))

In [51]:
tour_data1 = pd.read_excel('../input/tourism-data/train/2018-2019.xlsx', sheet_name=2)
tour_data2 = pd.read_excel('../input/tourism-data/train/2020-2021.xlsx', sheet_name=2)
tour_data = pd.concat([tour_data1, tour_data2], ignore_index=True)

def tour_counter(entity, data):
    return len(data[data['正文'].str.contains(entity)].index)

entity_tour = []
for entity in entities:
    entity_tour.append(tour_counter(entity, tour_data))

In [52]:
len(entities), len(entity_types), len(entity_evaluate), len(entity_essay), len(entity_tour)

(2181, 2181, 2181, 2181, 2181)

In [53]:
task2_2 = pd.DataFrame({'ID': ['ID' + str(x) for x in range(len(entities))],
                        'Type': entity_types,
                        'Name': entities,
                        'Evaluate': entity_evaluate,
                        'Essay': entity_essay,
                        'Tour': entity_tour})
task2_2

,ID,Type,Name,Evaluate,Essay,Tour
0,ID0,景区,茂名石化,7,0,0
1,ID1,酒店,茂名中心广场,1,0,0
2,ID2,景区,人民广场,15,49,3
3,ID3,景区,茂名高铁站,6,0,0
4,ID4,酒店,茂名站,29,0,6
...,...,...,...,...,...,...
2176,ID2176,餐饮,东汇城商场,1,0,0
2177,ID2177,餐饮,泰哥,2,0,0
2178,ID2178,餐饮,乌鸡,6,2,0
2179,ID2179,景区,三岔路口路,1,0,0


In [54]:
task2_2['Score'] = (task2_2['Evaluate'] + task2_2['Essay'] + task2_2['Tour']) / 3
minval, maxval = task2_2['Score'].min(), task2_2['Score'].max()
task2_2['Score'] = task2_2['Score'].apply(lambda x: (x - minval) / (maxval - minval))
task2_2.drop(columns=['Evaluate', 'Essay', 'Tour'], inplace=True)
task2_2.sort_values(by='Score' , inplace=True, ascending=False)
task2_2.columns = ['产品ID', '产品类型', '产品名称', '产品热度']
task2_2.to_csv('result2-2.csv', index=False)